In [25]:
# Libraries
from typing import Dict, List
import numpy as np
import casadi as ca

In [26]:
class CasADiPhysicsSolver:
    def __init__(self, length: float = 1.0, n_nodes: int = 50):
        self.L = length
        self.n = n_nodes
        self.dx = length / (n_nodes - 1)
        self.x = np.linspace(0, length, n_nodes)
        self._build_symbolic_problem()

    def _build_symbolic_problem(self):
        T_sym = ca.MX.sym('T', self.n)
        k_sym = ca.MX.sym('k')
        T_left_sym = ca.MX.sym('T_left')
        T_right_sym = ca.MX.sym('T_right')
        q_gen_sym = ca.MX.sym('q_gen')

        obj = 0
        for i in range(1, self.n-1):
            d2T_dx2 = (T_sym[i+1] - 2*T_sym[i] + T_sym[i-1]) / (self.dx**2)
            residual = k_sym * d2T_dx2 + q_gen_sym
            obj += residual **2
        
        g = ca.vertcat(T_sym[0] - T_left_sym, T_sym[-1] - T_right_sym)
        p = ca.vertcat(k_sym, T_left_sym, T_right_sym, q_gen_sym)

        nlp = {
            'x': T_sym,
            'p': p,
            'f': obj,
            'g': g
        }

        opts = {
            'ipopt.print_level': 0,
            'print_time': 0,
            'ipopt.max_iter': 1000
        }

        self.solver = ca.nlpsol('solver', 'ipopt', nlp, opts)    
    
    def solve(self, k: float, T_left: float, T_right: float, q_gen: float) -> np.ndarray:
        T0 = np.linspace(T_left, T_right, self.n)
        p = [k, T_left, T_right, q_gen]
        sol = self.solver(x0=T0, p=p, lbg=0, ubg =0)
        return np.array(sol['x']).flatten()

In [27]:
def generate_experimental_data(n_samples: int = 200, n_nodes: int = 50,
                               noise_level: float = 0.5) -> List[Dict]:
    """Generate synthetic experimental data"""
    physics_solver = CasADiPhysicsSolver(length=1.0, n_nodes=n_nodes)
    exp_data = []

    for i in range(n_samples):
        k_true = np.random.uniform(10, 80)
        T_left = np.random.uniform(350, 450)
        T_right = T_left + np.random.uniform(50, 100)
        q_gen = np.random.uniform(0, 800)

        T_true = physics_solver.solve(k_true, T_left, T_right, q_gen)
        T_measured = T_true + np.random.normal(0, noise_level, n_nodes)

        exp_data.append({
            'T_measured': T_measured,
            'T_left': T_left,
            'T_right': T_right,
            'q_gen': q_gen,
            'k_true': k_true
        })

        if (i + 1) % 50 == 0:
            print(f"  Generated {i + 1}/{n_samples} samples")

    return exp_data            
        

In [28]:
# main
n_nodes = 50
n_samples = 200
noise_level = 0.5

# step 1: generate data
exp_data = generate_experimental_data(n_samples, n_nodes, noise_level)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

  Generated 50/200 samples
  Generated 100/200 samples
  Generated 150/200 samples
  Generated 200/200 samples


In [29]:
print(exp_data)

[{'T_measured': array([431.42103583, 432.63069441, 434.94508145, 437.53275958,
       438.38014923, 441.30071531, 442.83712225, 445.06608471,
       447.17406418, 448.25911453, 451.51931867, 452.87978331,
       455.31158373, 456.71908675, 459.59680869, 461.28268516,
       464.51551852, 466.27530848, 468.50413617, 469.82969635,
       472.0185253 , 473.86884215, 475.84298037, 477.42184296,
       480.24401572, 481.95240297, 484.03524087, 484.41641039,
       487.91148166, 489.63643743, 491.77379653, 493.44333124,
       495.63409309, 498.6552485 , 499.83703639, 501.3488943 ,
       503.23891634, 506.39926144, 507.74472436, 509.81071688,
       511.81334813, 513.20492977, 516.08631257, 518.58296517,
       519.25515924, 521.48845483, 524.29671157, 525.79769272,
       528.00591308, 529.63495934]), 'T_left': 430.56301770945635, 'T_right': 529.3794505470365, 'q_gen': 459.1114935477501, 'k_true': 66.78836825653345}, {'T_measured': array([415.93592356, 418.76866686, 420.06530857, 421.74261